# Coursework Preproccessing

Student ID: B721581 | Data Completed: 22/01/23

The following is a Python program that creates SQLite database tables for a student attendance tracking system. The program does the following:
* Reads data from csv files in a given directory
* Cleans  and formats the data appropriately  
* Stores the data into the SQL tables

## Imports

In [13]:
import pandas as pd
import numpy as np
import sqlite3
import os

## Functions

In [14]:
def read_data():
    """
    Reads the data from each csv file in the directory
    into a dictionary of dataframes.
    
    input: None
    output: returns a dictionary of the csv files
    
    directory = folder name
    att_data = attendance data dictionary
    """
    
    # define local variables and collections
    directory = 'cop504cwdata'
    att_data = {}
    
    # loop through files in the directory 
    # and add data to dictionary of dataframes
    for i in os.listdir(directory):
        att_data[i] = pd.read_csv(directory + '/' + i)
        df_name = ('df' + i[:8])
        att_data[df_name] = att_data.pop(i)

    return att_data


    
def remove_na(att_data):
    """
    For each dataframe in the dictionary:
        - it replaces certain cells with appropriate values
        - it removes rows and columns with all NA values
    
    input: attendance data dictionary
    output: returns cleaned attendance data dictionary
    
    att_data = attendance data dictionary
    clean_att_data = clean attendance data dictionary
    Ex_replace = value to replace Ex
    GPS_replace = value to replace GPS
    X_replace = value to replace X
    """
    
    # define local variables and collections
    clean_att_data = {}
    Ex_replace = np.nan
    GPS_replace = True
    X_replace = False

    # loop through att_data dictionary
    for name, df in att_data.items():

        # replace Ex, GPS and X values
        att_data[name].replace(['Ex', 'GPS', 'X'], 
                               [Ex_replace, GPS_replace, X_replace], 
                               inplace = True)

        # remove rows and columns with all NA values
        cols = list(att_data[name].columns[1:])
        att_data[name].dropna(axis = 0, subset = cols, 
                              how = 'all', inplace = True)
        att_data[name].dropna(axis = 1, how = 'all', inplace = True)
        
        clean_att_data[name] = df

    return clean_att_data

    
    
def create_ses_df(att_data):    
    """
    For each dataframe in the dictionary it creates a new dataframe 
    of each session using the column headings.
    
    input: attendance data dictionary
    output: returns sessions data dictionary
    
    att_data = attendance data dictionary
    ses_data = individual sessions dataframe
    sessions_data = sessions data dictionary
    """
    
    # define local variables and collections
    sessions_data = {}
    
    # loop through att_data dictionary
    for name, df in att_data.items():

        # create list of column headings
        ses_data = att_data[name].columns.tolist()
        ses_data.pop(0)

        # create dataframe from list
        ses_data = pd.DataFrame({'col':ses_data})
        ses_data = ses_data['col'].str.split(pat = '\n', expand = True)

        # split specific columns 
        ses_data[['weekday', 'date']] = ses_data[1].str.split(expand = True)
        ses_data[['start_time', '-', 'end_time']] = ses_data[2]\
                                                    .str.split(expand = True)
        ses_data[['semester', 'week']] = ses_data[0].str.split(pat = '.', 
                                                               expand = True)

        # remove unnecessary columns
        ses_data.drop(columns = [0, 1, 2, 5, '-'], inplace = True)

        # rename and rearrange columns 
        ses_data['date'] = ses_data['date'].str[1:-1]
        ses_data = ses_data.rename(columns = {3:'type', 4:'room'})
        ses_data = ses_data[['semester', 'week', 'weekday', 'date', 
                             'start_time', 'end_time', 'type', 'room']]

        sessions_data[name] = ses_data

    return sessions_data



def rename_cols(clean_att_data):
    """
    Renames the columns of each dataframe in the dictionary.
    
    input: clean attendance data dictionary
    output: returns cleaner attendance data dictionary
    
    clean_att_data = clean attendance data dictionary
    col = individual column headings
    col_name_lst = list of new column headings
    """
    
    # loop through clean_att_data dictionary
    for name, df in clean_att_data.items():
        col_name_lst = []

        # loop through columns in each dataframe and append to col_name_lst
        for index in range(clean_att_data[name].shape[1]):
            col = clean_att_data[name].iloc[:, index]

            if col.name[0] == 's':
                col.name = 'sid'
                col_name_lst.append(col.name)

            elif col.name[0] == 'S':
                col.name = (col.name[:5] + '_' + str(index))            
                col_name_lst.append(col.name)

            else:
                continue

        clean_att_data[name].columns = col_name_lst
        clean_att_data[name].reset_index(drop = True, inplace = True)
        clean_att_data[name] = df

    return clean_att_data



def write_to_db(clean_att_data, sessions_data):
    """
    Saves each dataframe in each dictionary as a table in the SQL database.
    
    input: cleaner attendance data dictionary
           & sessions data dictionary
    output: prints message & displays list
    
    clean_att_data = cleaner attendance data dictionary
    sessions_data = sessions data dictionary
    conn = connection to SQL database
    db_name = name of database
    module = list of modules
    """
    
    # define local variables and collections
    db_name = 'CWDatabase'
    
    conn = sqlite3.connect(db_name + '.db')

    # loop through clean_att_data dictionary and add dataframes to SQL database
    for name, df in clean_att_data.items():
        df.to_sql(name[4:]+'_att_data', con = conn, if_exists = 'replace')

    # loop through sessions_data dictionary and add dataframes to SQL database
    for name, df in sessions_data.items():
        df.to_sql(name[4:]+'_ses_data', con = conn, if_exists = 'replace')

    conn.close()

    modules = [item[4:] for item in list(clean_att_data.keys())]

    print('Successfully created SQLite database for:')
    display(modules)

## Main Function

In [15]:
def preprocess():
    """
    Combines the above functions into one main function:
        - Reads the csv files in the directory, 
          cleans the data and saves it to the SQL database
    
    input: None
    output: None
    
    att_data = attendance data dictionary
    clean_att_data = clean attendance data dictionary
    ses_data = sessions data dictionary
    clean_att_data_2 = cleaner attendance data dictionary
    """
    
    att_data = read_data()
    
    clean_att_data = remove_na(att_data)
    
    ses_data = create_ses_df(clean_att_data)
    
    clean_att_data_2 = rename_cols(clean_att_data)
    
    write_to_db(clean_att_data_2, ses_data)

## Function Demo

In [16]:
preprocess()

Successfully created SQLite database for:


['COA122', 'COA111']